In [2]:
import pandas as pd

In [3]:
import pickle as pkl
#with open('data/unmatched_closings.pkl', 'rb') as f:
 #   pw_closings = pkl.load(f)

#pw_closings.head()

In [3]:
#pw_closings.columns

In [4]:
cincy = pd.read_csv("data/Cincy_All_Listings_Closings_Only_2024_08_30.csv")
#lb_buildings = pd.read_csv("data/lightbox_buildings/cincy.csv")
#for i in range(2, 14):
 #       lb_buildings = pd.concat(
  #          [lb_buildings, pd.read_csv(f"data/lightbox_buildings/cincy_{i}.csv")]
   #     )
#cincy = cincy.merge(lb_buildings, left_on="listing_key", right_on="listingkey")
cincy = cincy[['id', 'rebny_id', 'listing_key', 'address', 'zip','sold_price', 'last_event']]

/var/folders/6l/yzmcq93j41zf3wrth_tyl6g80000gn/T/ipykernel_27122/2673371135.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  cincy = pd.read_csv("data/Cincy_All_Listings_Closings_Only_2024_08_30.csv")


In [5]:
print(len(cincy['id']), len(cincy['id'].unique()))

760825 756939


In [6]:
print(len(cincy['rebny_id']), len(cincy['rebny_id'].unique()))

760825 755225


In [4]:
import os
def load_lightbox_data(fpath: str, service_zips: list) -> pd.DataFrame:
    df = pd.read_csv(fpath)
    df = df[
        [
            "DEED_LID",
            "ASSESSMENT_LID",
            "SITE_ADDR",
            "SITE_ZIP",
            "DATE_TRANSFER",
            "DATE_FILING",
            "VAL_TRANSFER",
        ]
    ]
    df = df[df["SITE_ZIP"].isin(service_zips)]
    df.dropna(subset="SITE_ADDR", inplace=True)
    df.dropna(subset="VAL_TRANSFER", inplace=True)
    return df[~(df["SITE_ADDR"] == "N A")]

def get_lightbox_files(service_zips: list, state: str):
    lb_dirs = []
    for root, dirs, files in os.walk(f"data/"):
        lb_dirs.extend(dirs)
    # for state in states:
    deeds_dirs = [x for x in lb_dirs if state in x and "HistoricalTransaction" in x]
    # print(states, deeds_dirs)
    deeds_df = load_lightbox_data(
        f"data/{deeds_dirs[0]}/Deed_{state}.csv", service_zips
    )
    return deeds_df

In [8]:
with open('data/matches_buckets_OH.pkl', 'rb') as f:
    matches = pkl.load(f)
with open('data/matches_buckets_IN.pkl', 'rb') as f:
    matches_in = pkl.load(f)
for k in matches:
    matches[k] = pd.concat([matches[k], matches_in[k]])
# matches

In [9]:
matches.keys()

dict_keys([((datetime.timedelta(0), datetime.timedelta(days=14)), (0, 0.1)), ((datetime.timedelta(days=14), datetime.timedelta(days=30)), (0, 0.1)), ((datetime.timedelta(days=30), datetime.timedelta(days=90)), (0, 0.1)), ((datetime.timedelta(days=90), datetime.timedelta(days=180)), (0, 0.1)), ((datetime.timedelta(days=180), datetime.timedelta(days=365)), (0, 0.1)), ((datetime.timedelta(days=365), datetime.timedelta(days=730)), (0, 0.1)), ((datetime.timedelta(days=730), None), (0, 0.1))])

In [5]:
service_zips = pd.read_csv("data/cincy_service_zips_2024_08_28.csv").dropna()["zip"]

In [11]:
from tqdm.auto import tqdm
for state in tqdm(['OH', 'IN']):
    deeds_df = get_lightbox_files(service_zips, state)
    for k in matches:
        if len(matches[k])>0:
            matches[k].merge(cincy, on='id').merge(deeds_df, on='DEED_LID').groupby('type_of_match').sample(frac=0.2).to_csv(f'data/output/cincy_OMS_match_check_{state}_{k}.csv', index=False)

  0%|          | 0/2 [00:00<?, ?it/s]

/var/folders/6l/yzmcq93j41zf3wrth_tyl6g80000gn/T/ipykernel_27122/321749638.py:3: DtypeWarning: Columns (5,6,7,8,9,10,11,14,18,19,20,23,24,26,27,48,58,59,63,64,65,66,67,70,71,73,95,96,97,98,100,115,116,117,120,146,148,153,154,155) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fpath)
/var/folders/6l/yzmcq93j41zf3wrth_tyl6g80000gn/T/ipykernel_27122/321749638.py:3: DtypeWarning: Columns (5,6,7,8,9,10,11,18,19,20,23,25,26,27,48,53,58,59,60,61,63,64,65,66,67,70,71,95,96,97,98,100,115,117,120,146,148,153,154,155) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fpath)


In [6]:
oms = pd.read_csv('data/off_market_sales_OH.csv')
len(oms)

/var/folders/6l/yzmcq93j41zf3wrth_tyl6g80000gn/T/ipykernel_16414/2536317230.py:1: DtypeWarning: Columns (1,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  oms = pd.read_csv('data/off_market_sales_OH.csv')


6405748

In [13]:
import pickle as pkl
with open('data/output/threshold_matches_buckets_OH.pkl', 'rb') as f:
    check = pkl.load(f)

deeds_matched = []
for k in check.keys():
    deeds_matched += list(check[k]['DEED_LID'])
len(deeds_matched)

66759

In [12]:
oms[oms['DEED_LID'].isin(check[list(check.keys())[0]]['DEED_LID'])]

,DEED_LID,ASSESSMENT_LID,DATE_TRANSFER,SITE_ADDR,SITE_HOUSE_NUMBER,SITE_DIRECTION,SITE_STREET_NAME,SITE_MODE,SITE_UNIT_NUMBER,SITE_CITY,SITE_STATE,SITE_ZIP,SITE_PLUS_4,DATE_FILING,VAL_TRANSFER,LONGITUDE,LATITUDE,normalized_address
3991,0400AYLHPEU8FNR420BFF2,NaN,2006-06-07,5300 STEAM FURNACE RD,NaN,NaN,NaN,NaN,NaN,PEEBLES,OH,45660.0,9508.0,2006-06-01 00:00:00,131250.0,NaN,NaN,5300 AM FURNACE RD
4802,04040A5BE9MDEXDC7CQ54Q,NaN,2008-08-11,2419 B HOOP RIDGE RO,NaN,NaN,NaN,NaN,NaN,PEEBLES,OH,45660.0,NaN,2008-08-06 00:00:00,70000.0,NaN,NaN,2419 B HOOP RDG RO
4888,0400QQ8KIXFU0GBN6O8OF1,NaN,1995-01-05,HARRIS AVE,NaN,NaN,NaN,NaN,NaN,BETHEL,OH,45106.0,NaN,1995-01-05 00:00:00,99000.0,NaN,NaN,HARRIS AVE
4904,0401886TL1UXN66Z95XKDX,NaN,1999-05-05,205 N 2ND ST,NaN,NaN,NaN,NaN,NaN,LOVELAND,OH,45140.0,2953.0,1999-05-05 00:00:00,150000.0,NaN,NaN,205 N 2ND ST
4905,0406XRD6I7G48YK130IYSU,NaN,1999-05-05,1ST ST,NaN,NaN,NaN,NaN,NaN,LOVELAND,OH,45140.0,NaN,1999-05-05 00:00:00,150000.0,NaN,NaN,1ST ST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6055929,04002VV9HQOI5P67XZTQ5K,0307L75NQQM8IX61DN1I8A,2015-07-16,1949 18TH ST SW,1949,NaN,18TH,ST,NaN,AKRON,OH,44314.0,2870.0,2015-07-14 00:00:00,111000.0,-81.562012,41.050764,1949 18TH ST SW
6082739,0406ULIEIX5LRT95RV2YOD,0302A5WNSHZ0EWS8QAOG8A,2008-03-05,2162 TIBBETTS WICK RD,2162,NaN,TIBBETTS WICK,RD,NaN,GIRARD,OH,44420.0,1216.0,2008-03-04 00:00:00,116000.0,-80.646877,41.179720,2162 TIBBETTS WICK RD
6132791,0404N56CLJETHJINRIKG35,03013JP9UKYMTV0QX8IVQ5,2019-02-21,1503 DIFFORD DR,1503,NaN,DIFFORD,DR,NaN,NILES,OH,44446.0,2831.0,2019-02-15 00:00:00,103000.0,-80.751371,41.201204,1503 DIFFORD DR
6189176,0404CY2L8M2FWNB0FL3UW1,0301UGSR1Q6MALZOI06LOT,2012-07-23,1006 BERWIN ST,1006,NaN,BERWIN,ST,NaN,AKRON,OH,44310.0,1433.0,2012-07-18 00:00:00,34000.0,-81.511728,41.112426,1006 BERWIN ST


In [15]:
matches = pd.read_csv('data/output/matched_deeds_with_closings_OH.csv')

In [15]:
len(matches)

779898

In [16]:
len(oms)

6405748

In [17]:
matches.head()

,id,DEED_LID,type_of_match
0,7753413,0401J1TZMAZ2Z4I5Z6IU3N,0_parcel_match
1,7753413,0400POAGZO60T6P9OB6YQM,0_parcel_match
2,7753413,0403L38CF4IDB1VZQP6P3Q,0_parcel_match
3,7753413,0403L38CF4IDB1VZQP6P3Q,0_parcel_match
4,7753413,04018F921GVA919MZJWTWV,0_parcel_match


In [18]:
oms.head()

,DEED_LID,ASSESSMENT_LID,DATE_TRANSFER,SITE_ADDR,SITE_HOUSE_NUMBER,SITE_DIRECTION,SITE_STREET_NAME,SITE_MODE,SITE_UNIT_NUMBER,SITE_CITY,SITE_STATE,SITE_ZIP,SITE_PLUS_4,DATE_FILING,VAL_TRANSFER,LONGITUDE,LATITUDE,normalized_address
0,0405FZTMTFH5JJI31OUKV8,NaN,2007-05-14,1188 GINKO DR,NaN,NaN,NaN,NaN,NaN,WEST SALEM,OH,44287.0,9138.0,2007-04-05 00:00:00,50000.0,NaN,NaN,1188 GINKO DR
1,0401Y9QMYVIQRSU1H7F7C0,NaN,1995-01-03,COON ROAD,NaN,NaN,NaN,NaN,NaN,LIMA,OH,45806.0,NaN,1995-01-03 00:00:00,5000.0,NaN,NaN,COON RD
2,0407BHZKR3TRICFC2Y8HHH,NaN,1995-02-13,"LINCOLN HWY, E",NaN,NaN,NaN,NaN,NaN,ELIDA,OH,45807.0,NaN,1995-02-13 00:00:00,25250.0,NaN,NaN,"LINCOLN HWY, E"
3,0401CM68JE6YJNGWYIMZ7P,NaN,1995-01-03,MADDEN ROAD,NaN,NaN,NaN,NaN,NaN,LIMA,OH,45806.0,NaN,1995-01-03 00:00:00,12500.0,NaN,NaN,MADDEN RD
4,04001JFOEP7Y3EVKVLM179,NaN,1995-01-04,MADDEN ROAD,NaN,NaN,NaN,NaN,NaN,LIMA,OH,45806.0,NaN,1995-01-04 00:00:00,13000.0,NaN,NaN,MADDEN RD


In [20]:
oms[~oms['DEED_LID'].isin(matches['DEED_LID'])]

,DEED_LID,ASSESSMENT_LID,DATE_TRANSFER,SITE_ADDR,SITE_HOUSE_NUMBER,SITE_DIRECTION,SITE_STREET_NAME,SITE_MODE,SITE_UNIT_NUMBER,SITE_CITY,SITE_STATE,SITE_ZIP,SITE_PLUS_4,DATE_FILING,VAL_TRANSFER,LONGITUDE,LATITUDE,normalized_address
0,0405FZTMTFH5JJI31OUKV8,NaN,2007-05-14,1188 GINKO DR,NaN,NaN,NaN,NaN,NaN,WEST SALEM,OH,44287.0,9138.0,2007-04-05 00:00:00,50000.0,NaN,NaN,1188 GINKO DR
1,0401Y9QMYVIQRSU1H7F7C0,NaN,1995-01-03,COON ROAD,NaN,NaN,NaN,NaN,NaN,LIMA,OH,45806.0,NaN,1995-01-03 00:00:00,5000.0,NaN,NaN,COON RD
2,0407BHZKR3TRICFC2Y8HHH,NaN,1995-02-13,"LINCOLN HWY, E",NaN,NaN,NaN,NaN,NaN,ELIDA,OH,45807.0,NaN,1995-02-13 00:00:00,25250.0,NaN,NaN,"LINCOLN HWY, E"
3,0401CM68JE6YJNGWYIMZ7P,NaN,1995-01-03,MADDEN ROAD,NaN,NaN,NaN,NaN,NaN,LIMA,OH,45806.0,NaN,1995-01-03 00:00:00,12500.0,NaN,NaN,MADDEN RD
4,04001JFOEP7Y3EVKVLM179,NaN,1995-01-04,MADDEN ROAD,NaN,NaN,NaN,NaN,NaN,LIMA,OH,45806.0,NaN,1995-01-04 00:00:00,13000.0,NaN,NaN,MADDEN RD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405743,0403441RT7S06D38CLZFHL,03007FBB54UZQ1O4S0RDX7,2016-01-05,1846 WINDY RIDGE RD,1846.0,NaN,WINDY RIDGE,RD,NaN,VINCENT,OH,45784.0,5230.0,2015-12-30 00:00:00,190000.0,-81.724803,39.354738,1846 WINDY RDG RD
6405744,0400YS7Y6DBBAQ4Z5FPVTO,0307EU2GCR9OOPTZJASPZU,2022-02-07,CURRY RD,NaN,NaN,CURRY,RD,NaN,NaN,NaN,43153.0,NaN,2022-01-19 00:00:00,118100.0,-83.653143,39.717350,CURRY RD
6405745,0402FWJPLTXEPC1CRBRNBX,03004VWGDU0IZCLN1RQHEI,2016-10-17,308 LAUREL HILLS DR,308.0,NaN,LAUREL HILLS,DR,NaN,STEUBENVILLE,OH,43952.0,7016.0,2016-10-11 00:00:00,290000.0,-80.687247,40.400693,308 LAUREL HLS DR
6405746,0402MD7BP7WNLBHO8C8MTV,03003MJ9WXIX5FJ2H6NSWM,2023-06-23,PVT DR 4442 CO RD 2,NaN,NaN,PVT DR 4442 CO RD 2,NaN,NaN,NaN,OH,45619.0,NaN,2023-06-17 00:00:00,8300.0,-82.420969,38.511534,PVT DR 4442 CO RD 2


In [22]:
matches.head()#4320 Hengehold Dr

,id,DEED_LID,type_of_match
0,7753413,0401J1TZMAZ2Z4I5Z6IU3N,0_parcel_match
1,7753413,0400POAGZO60T6P9OB6YQM,0_parcel_match
2,7753413,0403L38CF4IDB1VZQP6P3Q,0_parcel_match
3,7753413,0403L38CF4IDB1VZQP6P3Q,0_parcel_match
4,7753413,04018F921GVA919MZJWTWV,0_parcel_match


In [28]:
oms[oms['DEED_LID']=='0400D3P5KWGDHJV427TBUE']

,DEED_LID,ASSESSMENT_LID,DATE_TRANSFER,SITE_ADDR,SITE_HOUSE_NUMBER,SITE_DIRECTION,SITE_STREET_NAME,SITE_MODE,SITE_UNIT_NUMBER,SITE_CITY,SITE_STATE,SITE_ZIP,SITE_PLUS_4,DATE_FILING,VAL_TRANSFER,LONGITUDE,LATITUDE,normalized_address


In [29]:
len(oms)

6405748

In [7]:
oms[oms['SITE_ADDR'].str.lower() == '5296 belclare rd']

,DEED_LID,ASSESSMENT_LID,DATE_TRANSFER,SITE_ADDR,SITE_HOUSE_NUMBER,SITE_DIRECTION,SITE_STREET_NAME,SITE_MODE,SITE_UNIT_NUMBER,SITE_CITY,SITE_STATE,SITE_ZIP,SITE_PLUS_4,DATE_FILING,VAL_TRANSFER,LONGITUDE,LATITUDE,normalized_address


In [14]:
oms[oms['SITE_ADDR'].str.lower().str.contains('belclare')]['SITE_ADDR'].unique()

array(['5257 BELCLARE RD', '5290 BELCLARE RD', '5266 BELCLARE RD',
       '5276 BELCLARE RD', '5282 BELCLARE RD', '5263 BELCLARE RD',
       '5286 BELCLARE RD', '5283 BELCLARE RD', '5289 BELCLARE RD',
       '5320 BELCLARE RD', '5354 BELCLARE RD', '5362 BELCLARE RD',
       '5370 BELCLARE RD', '5378 BELCLARE RD', '5347 BELCLARE RD',
       '5396 BELCLARE RD', '5397 BELCLARE RD', '5295 BELCLARE RD',
       '5228 BELCLARE RD', '5240 BELCLARE RD', '5294 BELCLARE RD',
       '5273 BELCLARE RD', '5208 BELCLARE RD APT 1',
       '5208 BELCLARE RD APT 2', '5208 BELCLARE RD APT 3',
       '5208 BELCLARE RD APT 4', '5208 BELCLARE RD APT 6',
       '5269 BELCLARE RD', '5208 BELCLARE RD APT 7',
       '5208 BELCLARE RD APT 8', '5208 BELCLARE RD APT 10',
       '5208 BELCLARE RD APT 12', '5210 BELCLARE RD APT 13',
       '5210 BELCLARE RD APT 14', '5210 BELCLARE RD APT 15',
       '5210 BELCLARE RD APT 16', '5210 BELCLARE RD APT 17',
       '5210 BELCLARE RD APT 18', '5210 BELCLARE RD APT 19',
   

In [16]:
matches.head()

,id,DEED_LID,type_of_match
0,7753413,0401J1TZMAZ2Z4I5Z6IU3N,0_parcel_match
1,7753413,0400POAGZO60T6P9OB6YQM,0_parcel_match
2,7753413,0403L38CF4IDB1VZQP6P3Q,0_parcel_match
3,7753413,0403L38CF4IDB1VZQP6P3Q,0_parcel_match
4,7753413,04018F921GVA919MZJWTWV,0_parcel_match


In [17]:
deeds_df = get_lightbox_files(service_zips, 'OH')
deeds_df.head()

/var/folders/6l/yzmcq93j41zf3wrth_tyl6g80000gn/T/ipykernel_16414/321749638.py:3: DtypeWarning: Columns (5,6,7,8,9,10,11,14,18,19,20,23,24,26,27,48,58,59,63,64,65,66,67,70,71,73,95,96,97,98,100,115,116,117,120,146,148,153,154,155) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fpath)


,DEED_LID,ASSESSMENT_LID,SITE_ADDR,SITE_ZIP,DATE_TRANSFER,DATE_FILING,VAL_TRANSFER
74,0405FZTMTFH5JJI31OUKV8,NaN,1188 GINKO DR,44287.0,2007-05-14 00:00:00,2007-04-05 00:00:00,50000.0
213,0401Y9QMYVIQRSU1H7F7C0,NaN,COON ROAD,45806.0,1995-01-03 00:00:00,1995-01-03 00:00:00,5000.0
214,0407BHZKR3TRICFC2Y8HHH,NaN,"LINCOLN HWY, E",45807.0,1995-02-13 00:00:00,1995-02-13 00:00:00,25250.0
215,0401CM68JE6YJNGWYIMZ7P,NaN,MADDEN ROAD,45806.0,1995-01-03 00:00:00,1995-01-03 00:00:00,12500.0
216,04001JFOEP7Y3EVKVLM179,NaN,MADDEN ROAD,45806.0,1995-01-04 00:00:00,1995-01-04 00:00:00,13000.0


In [18]:
deeds_df[deeds_df['SITE_ADDR'].str.lower() == '5296 belclare rd']

,DEED_LID,ASSESSMENT_LID,SITE_ADDR,SITE_ZIP,DATE_TRANSFER,DATE_FILING,VAL_TRANSFER
7129490,04057YU9BS1690DPYCIYJZ,0303XPNM3MZT7RKCX8C83U,5296 BELCLARE RD,45247.0,2000-07-03 00:00:00,2000-06-30 00:00:00,80000.0
7129491,04001F1R2OP068R238WC9J,0303XPNM3MZT7RKCX8C83U,5296 BELCLARE RD,45247.0,2001-10-25 00:00:00,2001-10-19 00:00:00,89000.0
7129492,0405GPNTNB3UZ3YPF35Y8Q,0303XPNM3MZT7RKCX8C83U,5296 BELCLARE RD,45247.0,2003-06-09 00:00:00,2003-05-29 00:00:00,98000.0


In [19]:
matches[matches['DEED_LID'].isin(['04057YU9BS1690DPYCIYJZ', '04001F1R2OP068R238WC9J', '0405GPNTNB3UZ3YPF35Y8Q'])]

,id,DEED_LID,type_of_match
483389,4525078,04057YU9BS1690DPYCIYJZ,0_parcel_match
483390,4943598,04001F1R2OP068R238WC9J,0_parcel_match
483391,4995983,0405GPNTNB3UZ3YPF35Y8Q,0_parcel_match
